In [ ]:
import 
filename = sys.argv[0]

In [41]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [42]:
import pandas as pd
df = pd.read_csv('../data/corpus_check_long.csv')

In [43]:
df

,siren,legal_name,id,url_article,corpus
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,Ipsen lorgne les peptides de PeptiMimesis\n
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,Ipsen : accord important avec Probi\n \n\npubl...
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,La médecine générale d'Ipsen en panne au 1er t...
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
...,...,...,...,...,...
64289,803894872,MDP,76494,http://www.lyonpoleimmo.com/2019/10/22/56480/b...,Beynost : Maxon lance la réalisation de son ce...
64290,834289373,LAGARDERE MEDIA NEWS,76497,https://www.lexpress.fr/actualite/medias/herve...,Hervé Gattegno est nommé directeur général des...
64291,813956265,E-VALLEY SERVICES 2,76506,https://www.lavoixdunord.fr/654836/article/201...,\n Vendredi 08 novembre 2019 Consulter le jour...
64292,440088938,SENVION FRANCE,76507,https://www.greenunivers.com/2019/10/siemens-g...,Siemens Gamesa reprend les services de Senvion...


In [44]:
indexNames = []
def get_corrupt_data(df):
    for counter,data in enumerate(df.iterrows()):
        i, row = data
        tmp = df.corpus[i]
        if ("�") in tmp:
            indexNames.append(i)
get_corrupt_data(df)      
df.drop(indexNames , inplace=True)

## Filter : keep only companies that have at least 7 articles, and their 


In [45]:
#Build list of companies that have more then 7 articles in the corpora
top = df["siren"].value_counts()
top = top.where(top>=7).dropna()
topList = list(top.index)
df = df[df["siren"].isin(topList)]

## Filter: discard articles that are longer than 1,000,000 characters

In [46]:
df = df[df['corpus'].astype(str).map(len)<1000000]
df

,siren,legal_name,id,url_article,corpus
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,Ipsen lorgne les peptides de PeptiMimesis\n
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,Ipsen : accord important avec Probi\n \n\npubl...
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,La médecine générale d'Ipsen en panne au 1er t...
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
...,...,...,...,...,...
58371,489682005,ZENIKA,49051,https://emploibreizh.bzh/job/consultant-devops...,Website unavailable\n \n ...
58372,489682005,ZENIKA,53702,https://emploibreizh.bzh/job/developpeur-front...,Website unavailable\n \n ...
58373,489682005,ZENIKA,57506,https://www.fusacq.com/buzz/isatis-capital-ent...,"ISATIS CAPITAL entre au capital de ZENIKA, FUS..."
58374,489682005,ZENIKA,71582,https://uxjobs.fr/job/3428-ux-designer-h-f-lyon,[CDI] UX Designer H/F Lyon - Zenika - UXJobs.f...


## Filter: discard articles that are longer than 100 words


In [2]:
import re
import contractions
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr_core_news_sm')
nlp.max_length = 68111459
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
translator = str.maketrans('', ' ', string.punctuation)
print(string.punctuation)

2020-05-14 14:54:06,760 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-05-14 14:54:06,761 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-05-14 14:54:07,226 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [48]:
def cleaning(doc):
    doc = doc.replace('\n', ' ')
    doc = doc.replace('\r\n', ' ')
    doc = doc.replace('\r', ' ')
    doc = doc.replace('\t', ' ')
    return doc 
def remove_numbers(doc):
    doc = re.sub("\d+", "", doc)
    doc = doc.replace('m€', '')
    doc = doc.replace('k€', '')   
    return doc

In [53]:

# Tokenize text
def preprocessing(doc,train=False):        
        # Remove «»
        doc = doc.replace("«", " ")
        doc = doc.replace("»", " ")

        # To lowercase 
        doc = doc.lower()
        
        # Remove url's
        doc = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', doc, flags=re.MULTILINE)
        
        # Cleaning
        doc = cleaning(doc)
        
        # Remove numbers
        doc = remove_numbers(doc)
        
    
        # Remove multiple wite spaces 
        doc = re.sub(' +', ' ',doc)
        
        # Remove unicode breaking character
        doc = doc.replace(u'\xa0', u' ')
        
        if train: 
            result = []
            sentences = sent_tokenize(doc)
            for sent in sentences: 
                   # Remove punctuation
                sent = sent.translate(translator)
                sent += "\n"
                result.append(sent)
            return "".join(result)
        else:
            doc += "\n"
            return doc 

def preprocess_and_write_to_file(dataframe,train,index=0):
    f = codecs.open("dataEval" + str(index) + '.txt' , 'w', 'utf-8')
    for counter,data in enumerate(dataframe.iterrows()):
        i, row = data
        if(counter%5000==0):
            print("Thread " + str(index) + "processed " + str(counter) + "/" + str(dataframe.count()))
        preprocessed_text = preprocessing((row['corpus']),train)
        f.write(preprocessed_text)  # python will convert \n to os.linesep
    f.close()  

In [54]:
%%time
train = True
import codecs
import multiprocessing
import numpy as np
chunks = np.array_split(df,3)
manager = multiprocessing.Manager()
threads = []
for index,chunk in enumerate(chunks):
    thread = multiprocessing.Process(target=preprocess_and_write_to_file, args=(chunk,train,index))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()

Thread 0processed 0/siren          8510
legal_name     8510
id             8510
url_article    8510
corpus         8510
dtype: int64
Thread 1processed 0/siren          8509
legal_name     8509
id             8509
url_article    8509
corpus         8509
dtype: int64
Thread 2processed 0/siren          8509
legal_name     8509
id             8509
url_article    8509
corpus         8509
dtype: int64
Thread 0processed 5000/siren          8510
legal_name     8510
id             8510
url_article    8510
corpus         8510
dtype: int64
Thread 2processed 5000/siren          8509
legal_name     8509
id             8509
url_article    8509
corpus         8509
dtype: int64
Thread 1processed 5000/siren          8509
legal_name     8509
id             8509
url_article    8509
corpus         8509
dtype: int64
CPU times: user 37.3 ms, sys: 111 ms, total: 148 ms
Wall time: 17 s


In [55]:
! cat dataEval*.txt > evalFile
! cat dataEval*.txt > merge

In [52]:
def read_file(path):
    with open(path) as f:
        content = f.readlines()
    return content
    

In [29]:
data = read_file('evalFile')

In [32]:
df['preprocessedCorpus'] = data
del df['id']
del df['url_article']
del df['corpus']
df = df.rename({'preprocessedCorpus': 'corpus'}, axis='columns')


In [33]:
df

,siren,legal_name,corpus
0,419838529,IPSEN,ipsen lorgne les peptides de peptimimesis \n
1,419838529,IPSEN,ipsen accord important avec probi publié le à ...
2,419838529,IPSEN,la médecine générale dipsen en panne au er tri...
3,419838529,IPSEN,bourse en ligne information boursiere economie...
4,419838529,IPSEN,bourse en ligne information boursiere economie...
...,...,...,...
58371,489682005,ZENIKA,website unavailable this site is currently sus...
58372,489682005,ZENIKA,website unavailable this site is currently sus...
58373,489682005,ZENIKA,isatis capital entre au capital de zenika fusa...
58374,489682005,ZENIKA,cdi ux designer hf lyon zenika uxjobsfr tu te ...


In [37]:
df.to_csv('eval-without-lemm.csv')
df = pd.read_csv('eval-without-lemm.csv')

In [38]:
df

,Unnamed: 0,Unnamed: 0.1,siren,legal_name,corpus
0,0,0,419838529,IPSEN,ipsen lorgne les peptides de peptimimesis \n
1,1,1,419838529,IPSEN,ipsen accord important avec probi publié le à ...
2,2,2,419838529,IPSEN,la médecine générale dipsen en panne au er tri...
3,3,3,419838529,IPSEN,bourse en ligne information boursiere economie...
4,4,4,419838529,IPSEN,bourse en ligne information boursiere economie...
...,...,...,...,...,...
25523,25523,58371,489682005,ZENIKA,website unavailable this site is currently sus...
25524,25524,58372,489682005,ZENIKA,website unavailable this site is currently sus...
25525,25525,58373,489682005,ZENIKA,isatis capital entre au capital de zenika fusa...
25526,25526,58374,489682005,ZENIKA,cdi ux designer hf lyon zenika uxjobsfr tu te ...
